In [9]:
!pip install nltk

  Using cached nltk-3.6.7-py3-none-any.whl (1.5 MB)
     |████████████████████████████████| 288 kB 4.7 MB/s eta 0:00:01


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import nltk
from nltk.tokenize import word_tokenize
import string
import pickle
from sklearn import model_selection
from sklearn.metrics import confusion_matrix

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marcuscorreia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# load data
news = pd.read_csv("Data/final_news.csv")

In [14]:
news.dropna(inplace=True)

In [15]:
news["corpus"] = news["Article_title"] +" " +news["Article_body"]

In [16]:
news.head(2)

,Unnamed: 0,Article_title,Article_body,Title_length,Body_length,Target,Number_of_tweets,corpus,corpus_length_original
0,4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,9.0,317.0,0.0,0.0,The Battle of New York: Why This Primary Matte...,205.0
1,13,"Trump takes on Cruz, but lightly","Killing Obama administration rules, dismantlin...",6.0,17.0,0.0,0.0,"Trump takes on Cruz, but lightly Killing Obama...",16.0


In [17]:
x = news["corpus"]
y = news["Target"]

In [18]:
# split data into train/test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.4,random_state=42)

In [19]:
#Tfidif vectorizer
vectorizor = TfidfVectorizer(stop_words = "english")

In [20]:
# fit and transform to dataset
vec_x_train = vectorizor.fit_transform(x_train)

In [22]:
vectorizor.transform(x_test)

<9833x78841 sparse matrix of type '<class 'numpy.float64'>'
	with 1506632 stored elements in Compressed Sparse Row format>

In [23]:
vectorizor.get_feature_names_out()

array(['00', '000', '0000', ..., 'zzn3bqnfsk', 'zzucqevt3m', 'zzzzzzzz'],
      dtype=object)

In [24]:
vec_x_test = vectorizor.transform(x_test)

In [25]:
vec_x_test = vectorizor.transform(x_test)

In [26]:
# create a logistic regression classifier
log_reg_Classifier = LogisticRegression()

In [27]:
# fit the data
log_reg_Classifier.fit(vec_x_train,y_train)

LogisticRegression()

In [ ]:
pickle.dump(vectorizor, open("Model_files/Vectorisors/log_reg_vec.pkl", 'wb'))

In [35]:
pickle.dump(log_reg_Classifier, open("Model_files/Models/log_reg_model.pkl", 'wb'))

In [28]:
# remove non ascii characters
def remove_non_ascii(word):
    new_word = word.encode("ascii","ignore").decode()
    return new_word

In [29]:
def clean_input(input):
    # remove punctuation and stop
    # load language model
    sp = spacy.load('en_core_web_sm')
    # import stop words
    all_stopwords = sp.Defaults.stop_words
    # import puncuation
    punc  = string.punctuation

    # replace any weird characters
    text = input
    remove_non_ascii(text)

    # remove puncuation
    new_text = text.translate(str.maketrans('', '', punc))

    # tokenize and make everything lower case
    text_tokens = word_tokenize(new_text.lower())

    # remove stop words
    tokens_without_sw= [word for word in text_tokens if not word in all_stopwords]

    # make list a string again
    tokens_without_sw = " ".join(tokens_without_sw)
    tokens_without_sw

    return str(tokens_without_sw)

In [30]:
# all are from abc news
# # this predicts 0
# test_string = """WA health authorities urge anyone with COVID symptoms to get tested and isolate, as state records seven new cases"""

# # this predicts 0
# test_string = """Passenger killed in fiery crash in Adelaide's north, police investigate shooting between cars"""

# # this predicts 0
# test_string = """Environmentalists vow to fight latest Kimberley fracking proposal to unearth Australia's largest oil resource"""

## this predicts 1
# test_string = """How Facebook Fact-checking Can Backfire""" 

## this predicts 1
test_string = """Hulk Sad! Tatiana Maslany Says She Is Not Playing She-Hulk""" 

In [32]:
test_string = test_string

In [33]:
# test on the test data (this is testing on multiple article titles at once)
log_reg_vec_m = vectorizor.transform(x_test.tolist())

# test on random article title
log_reg_vec = vectorizor.transform([test_string])

In [34]:
# predicted values of each test article
predictions_m = log_reg_Classifier.predict(log_reg_vec_m)

# predicted values of random test article
predictions = log_reg_Classifier.predict(log_reg_vec)[0]

In [35]:
# print out result
print(predictions_m)

# print out result
predictions

[0. 1. 1. ... 1. 0. 0.]


1.0

In [36]:
z = pd.DataFrame({"Predictions":predictions_m,"Actual":y_test})
z

,Predictions,Actual
2350,0.0,0.0
2756342,1.0,1.0
2754595,1.0,1.0
2754230,1.0,1.0
2754986,1.0,1.0
...,...,...
1836910,1.0,1.0
10854,0.0,0.0
1838237,1.0,1.0
9364,0.0,0.0


In [37]:
print(f"Training Data Score: {log_reg_Classifier.score(vec_x_train, y_train)}")
print(f"Testing Data Score: {log_reg_Classifier.score(vec_x_test, y_test)}")

Training Data Score: 0.981217792243016
Testing Data Score: 0.9701006813790298


In [38]:
# show how many are not correct
z[z.Predictions != z.Actual].shape

(294, 2)

In [39]:
#load vectoriser
LR_vec = pickle.load( open("Model_files/Vectorisors/log_reg_vec.pkl","rb"))
# load model
LR_model = pickle.load(open("Model_files/Models/log_reg_model.pkl", 'rb'))

In [40]:
# using accuracy
kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
scoring = 'accuracy'
results = model_selection.cross_val_score(LR_model, vec_x_train, y_train, cv=kfold, scoring=scoring)
print("Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

Accuracy: 0.969 (0.005)


In [41]:
# using roc
kfold = model_selection.KFold(n_splits=10, random_state=7, shuffle=True)
scoring = 'roc_auc'
results = model_selection.cross_val_score(LR_model, vec_x_train, y_train, cv=kfold, scoring=scoring)
print("Accuracy: %.3f (%.3f)" % (results.mean(), results.std()))

Accuracy: 0.994 (0.002)


In [42]:
# using confusion matrix
predicted2 = LR_model.predict(vec_x_test)
matrix = confusion_matrix(y_test, predicted2)
print(matrix)

ValueError: X has 78841 features, but LogisticRegression is expecting 77705 features as input.

In [43]:
#Top 20 fake
top20_fake = sorted(zip(log_reg_Classifier.coef_[0], vectorizor.get_feature_names_out()), reverse=True)[:20]
top20_fake= pd.DataFrame(top20_fake, columns=["Contribution","Word"])
top20_fake.to_csv("Data/contribution_top20_fake.csv")

In [44]:
top20_fake

,Contribution,Word
0,6.059114,video
1,5.426573,image
2,5.093540,trump
3,5.079294,just
4,4.991160,featured
5,3.782185,read
6,3.599880,hillary
7,3.477646,like
8,3.400683,watch
9,3.317268,getty


In [45]:
#Top 20 real
top20_real = sorted(zip(log_reg_Classifier.coef_[0], vectorizor.get_feature_names_out()))[:20]
top20_real= pd.DataFrame(top20_real, columns=["Contribution","Word"])
top20_real.to_csv("Data/contribution_top20_words_real.csv")

In [46]:
top20_real

,Contribution,Word
0,-13.580728,reuters
1,-13.272205,said
2,-3.968982,tuesday
3,-3.662648,minister
4,-3.605983,thursday
5,-3.490588,monday
6,-3.403684,friday
7,-3.334349,wednesday
8,-2.984265,washington
9,-2.837504,government
